# Generative Adversarial Network (GAN) for MNIST Digits

In [1]:
# Import required libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Reshape, BatchNormalization, LeakyReLU, Dropout
import numpy as np
import matplotlib.pyplot as plt

C:\Users\swaro\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Load the MNIST dataset
(X_train, _), (_, _) = tf.keras.datasets.mnist.load_data()

In [3]:
# Normalize pixel values to the range [-1, 1] (for the generator's output activation)
X_train = (X_train - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=-1)  

In [4]:
# Define constants
LATENT_DIM = 100  # Size of the random noise vector
IMG_SHAPE = (28, 28, 1)  # Shape of MNIST images
BATCH_SIZE = 128
EPOCHS = 5000
SAVE_INTERVAL = 1000

In [5]:
# Define the generator model
def build_generator():
    model = Sequential([
        Dense(256, activation='relu', input_dim=LATENT_DIM),
        BatchNormalization(momentum=0.8),
        LeakyReLU(0.2),
        Dense(512, activation='relu'),
        BatchNormalization(momentum=0.8),
        LeakyReLU(0.2),
        Dense(1024, activation='relu'),
        BatchNormalization(momentum=0.8),
        LeakyReLU(0.2),
        Dense(np.prod(IMG_SHAPE), activation='tanh'),
        Reshape(IMG_SHAPE)
    ])
    return model


In [6]:
# Define the discriminator model
def build_discriminator():
    model = Sequential([
        Flatten(input_shape=IMG_SHAPE),
        Dense(512),
        LeakyReLU(0.2),
        Dropout(0.3),
        Dense(256),
        LeakyReLU(0.2),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    return model


In [7]:
# Build and compile the discriminator
discriminator = build_discriminator()
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
# Build the generator
generator = build_generator()

In [9]:
# Create the GAN by combining the generator and discriminator
z = tf.keras.layers.Input(shape=(LATENT_DIM,))
generated_img = generator(z)
discriminator.trainable = False  # Freeze the discriminator while training the GAN
validity = discriminator(generated_img)

In [10]:
# Define the GAN model
gan = Model(z, validity)
gan.compile(optimizer='adam', loss='binary_crossentropy')

In [12]:
# Function to train the GAN
def train_gan(epochs, batch_size, save_interval):
    half_batch = batch_size // 2

    for epoch in range(epochs):
        # Train the discriminator
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        real_imgs = X_train[idx]

        noise = np.random.normal(0, 1, (half_batch, LATENT_DIM))
        fake_imgs = generator.predict(noise)

        real_labels = np.ones((half_batch, 1))
        fake_labels = np.zeros((half_batch, 1))

        d_loss_real = discriminator.train_on_batch(real_imgs, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_imgs, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        noise = np.random.normal(0, 1, (batch_size, LATENT_DIM))
        valid_labels = np.ones((batch_size, 1))
        g_loss = gan.train_on_batch(noise, valid_labels)

        # Print progress
        print(f"{epoch + 1}/{epochs} [D loss: {d_loss[0]:.4f}, acc.: {100 * d_loss[1]:.2f}%] [G loss: {g_loss:.4f}]")

        # Save generated images at intervals
        if (epoch + 1) % save_interval == 0:
            save_images(epoch + 1)


In [13]:
# Function to save generated images
def save_images(epoch):
    noise = np.random.normal(0, 1, (16, LATENT_DIM))
    gen_imgs = generator.predict(noise)
    gen_imgs = 0.5 * gen_imgs + 0.5  # Rescale images to [0, 1]

    plt.figure(figsize=(10, 10))
    for i in range(16):
        plt.subplot(4, 4, i + 1)
        plt.imshow(gen_imgs[i, :, :, 0], cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(f"gan_generated_{epoch}.png")
    plt.close()

In [14]:
# Train the GAN
train_gan(EPOCHS, BATCH_SIZE, SAVE_INTERVAL)

2/2 [==============================] - 4s 19ms/step
1/5000 [D loss: 0.8990, acc.: 38.28%] [G loss: 0.8470]
2/2 [==============================] - 0s 12ms/step
2/5000 [D loss: 0.8041, acc.: 50.00%] [G loss: 0.0373]
2/2 [==============================] - 0s 11ms/step
3/5000 [D loss: 0.8520, acc.: 50.00%] [G loss: 0.0073]
2/2 [==============================] - 0s 16ms/step
4/5000 [D loss: 0.7267, acc.: 51.56%] [G loss: 0.0124]
2/2 [==============================] - 0s 17ms/step
5/5000 [D loss: 0.5277, acc.: 50.78%] [G loss: 0.1658]
2/2 [==============================] - 0s 11ms/step
6/5000 [D loss: 0.3425, acc.: 79.69%] [G loss: 2.2524]
2/2 [==============================] - 0s 12ms/step
7/5000 [D loss: 0.4290, acc.: 79.69%] [G loss: 1.5779]
2/2 [==============================] - 0s 11ms/step
8/5000 [D loss: 0.4523, acc.: 62.50%] [G loss: 1.0976]
2/2 [==============================] - 0s 13ms/step
9/5000 [D loss: 0.5606, acc.: 64.84%] [G loss: 2.0025]
2/2 [==============================] 

76/5000 [D loss: 0.0053, acc.: 100.00%] [G loss: 23.7395]
2/2 [==============================] - 0s 19ms/step
77/5000 [D loss: 0.0010, acc.: 100.00%] [G loss: 27.2796]
2/2 [==============================] - 0s 19ms/step
78/5000 [D loss: 0.0060, acc.: 100.00%] [G loss: 23.9661]
2/2 [==============================] - 0s 14ms/step
79/5000 [D loss: 0.0372, acc.: 98.44%] [G loss: 23.5098]
2/2 [==============================] - 0s 11ms/step
80/5000 [D loss: 0.0098, acc.: 100.00%] [G loss: 18.7940]
2/2 [==============================] - 0s 13ms/step
81/5000 [D loss: 0.0147, acc.: 100.00%] [G loss: 18.8290]
2/2 [==============================] - 0s 9ms/step
82/5000 [D loss: 0.0778, acc.: 96.09%] [G loss: 26.5927]
2/2 [==============================] - 0s 13ms/step
83/5000 [D loss: 0.0074, acc.: 100.00%] [G loss: 30.1670]
2/2 [==============================] - 0s 9ms/step
84/5000 [D loss: 0.4397, acc.: 82.03%] [G loss: 18.5678]
2/2 [==============================] - 0s 10ms/step
85/5000 [D loss

2/2 [==============================] - 0s 9ms/step
151/5000 [D loss: 0.0079, acc.: 100.00%] [G loss: 24.2690]
2/2 [==============================] - 0s 0s/step
152/5000 [D loss: 0.0056, acc.: 100.00%] [G loss: 23.6481]
2/2 [==============================] - 0s 14ms/step
153/5000 [D loss: 0.0017, acc.: 100.00%] [G loss: 23.2716]
2/2 [==============================] - 0s 10ms/step
154/5000 [D loss: 0.0016, acc.: 100.00%] [G loss: 22.3068]
2/2 [==============================] - 0s 8ms/step
155/5000 [D loss: 0.0042, acc.: 100.00%] [G loss: 21.8765]
2/2 [==============================] - 0s 11ms/step
156/5000 [D loss: 0.0008, acc.: 100.00%] [G loss: 22.0818]
2/2 [==============================] - 0s 16ms/step
157/5000 [D loss: 0.0030, acc.: 100.00%] [G loss: 21.7797]
2/2 [==============================] - 0s 14ms/step
158/5000 [D loss: 0.0286, acc.: 97.66%] [G loss: 22.1339]
2/2 [==============================] - 0s 4ms/step
159/5000 [D loss: 0.0006, acc.: 100.00%] [G loss: 22.7941]
2/2 [==

2/2 [==============================] - 0s 12ms/step
225/5000 [D loss: 0.0576, acc.: 98.44%] [G loss: 13.4477]
2/2 [==============================] - 0s 10ms/step
226/5000 [D loss: 0.0188, acc.: 99.22%] [G loss: 16.8895]
2/2 [==============================] - 0s 15ms/step
227/5000 [D loss: 0.0450, acc.: 99.22%] [G loss: 13.3377]
2/2 [==============================] - 0s 4ms/step
228/5000 [D loss: 0.0180, acc.: 100.00%] [G loss: 12.6639]
2/2 [==============================] - 0s 10ms/step
229/5000 [D loss: 0.0260, acc.: 99.22%] [G loss: 14.5527]
2/2 [==============================] - 0s 12ms/step
230/5000 [D loss: 0.0044, acc.: 100.00%] [G loss: 17.5682]
2/2 [==============================] - 0s 17ms/step
231/5000 [D loss: 0.0567, acc.: 97.66%] [G loss: 11.9666]
2/2 [==============================] - 0s 17ms/step
232/5000 [D loss: 0.1364, acc.: 96.09%] [G loss: 11.9281]
2/2 [==============================] - 0s 11ms/step
233/5000 [D loss: 0.0082, acc.: 100.00%] [G loss: 16.8465]
2/2 [===

2/2 [==============================] - 0s 9ms/step
300/5000 [D loss: 0.1412, acc.: 95.31%] [G loss: 7.5561]
2/2 [==============================] - 0s 8ms/step
301/5000 [D loss: 0.3263, acc.: 89.06%] [G loss: 6.2712]
2/2 [==============================] - 0s 13ms/step
302/5000 [D loss: 0.1001, acc.: 96.09%] [G loss: 8.3224]
2/2 [==============================] - 0s 7ms/step
303/5000 [D loss: 0.0914, acc.: 96.88%] [G loss: 10.0087]
2/2 [==============================] - 0s 12ms/step
304/5000 [D loss: 0.1365, acc.: 94.53%] [G loss: 9.1116]
2/2 [==============================] - 0s 5ms/step
305/5000 [D loss: 0.0438, acc.: 99.22%] [G loss: 7.4481]
2/2 [==============================] - 0s 10ms/step
306/5000 [D loss: 0.1300, acc.: 94.53%] [G loss: 7.2017]
2/2 [==============================] - 0s 6ms/step
307/5000 [D loss: 0.0505, acc.: 97.66%] [G loss: 9.5777]
2/2 [==============================] - 0s 8ms/step
308/5000 [D loss: 0.1056, acc.: 97.66%] [G loss: 7.7598]
2/2 [===================

...

2/2 [==============================] - 0s 10ms/step
4654/5000 [D loss: 0.4100, acc.: 78.91%] [G loss: 1.9555]
2/2 [==============================] - 0s 10ms/step
4655/5000 [D loss: 0.5364, acc.: 73.44%] [G loss: 1.8831]
2/2 [==============================] - 0s 18ms/step
4656/5000 [D loss: 0.4126, acc.: 78.12%] [G loss: 2.0000]
2/2 [==============================] - 0s 9ms/step
4657/5000 [D loss: 0.2716, acc.: 91.41%] [G loss: 2.3169]
2/2 [==============================] - 0s 8ms/step
4658/5000 [D loss: 0.5020, acc.: 73.44%] [G loss: 2.0721]
2/2 [==============================] - 0s 10ms/step
4659/5000 [D loss: 0.5214, acc.: 77.34%] [G loss: 1.8617]
2/2 [==============================] - 0s 19ms/step
4660/5000 [D loss: 0.4806, acc.: 77.34%] [G loss: 2.0460]
2/2 [==============================] - 0s 14ms/step
4661/5000 [D loss: 0.3476, acc.: 88.28%] [G loss: 2.0316]
2/2 [==============================] - 0s 11ms/step
4662/5000 [D loss: 0.4970, acc.: 75.78%] [G loss: 2.2822]
2/2 [=======

2/2 [==============================] - 0s 14ms/step
4729/5000 [D loss: 0.5006, acc.: 77.34%] [G loss: 2.0037]
2/2 [==============================] - 0s 16ms/step
4730/5000 [D loss: 0.5088, acc.: 75.78%] [G loss: 1.6764]
2/2 [==============================] - 0s 16ms/step
4731/5000 [D loss: 0.4673, acc.: 73.44%] [G loss: 1.5737]
2/2 [==============================] - 0s 16ms/step
4732/5000 [D loss: 0.4271, acc.: 81.25%] [G loss: 1.6750]
2/2 [==============================] - 0s 17ms/step
4733/5000 [D loss: 0.4329, acc.: 77.34%] [G loss: 1.8396]
2/2 [==============================] - 0s 10ms/step
4734/5000 [D loss: 0.6171, acc.: 71.09%] [G loss: 1.6494]
2/2 [==============================] - 0s 10ms/step
4735/5000 [D loss: 0.4943, acc.: 74.22%] [G loss: 1.8009]
2/2 [==============================] - 0s 15ms/step
4736/5000 [D loss: 0.3183, acc.: 86.72%] [G loss: 1.8285]
2/2 [==============================] - 0s 16ms/step
4737/5000 [D loss: 0.4714, acc.: 79.69%] [G loss: 1.8415]
2/2 [=====

2/2 [==============================] - 0s 10ms/step
4804/5000 [D loss: 0.4580, acc.: 75.78%] [G loss: 1.8865]
2/2 [==============================] - 0s 14ms/step
4805/5000 [D loss: 0.4348, acc.: 78.12%] [G loss: 1.8896]
2/2 [==============================] - 0s 10ms/step
4806/5000 [D loss: 0.4106, acc.: 82.03%] [G loss: 2.0003]
2/2 [==============================] - 0s 15ms/step
4807/5000 [D loss: 0.3767, acc.: 82.81%] [G loss: 1.9749]
2/2 [==============================] - 0s 11ms/step
4808/5000 [D loss: 0.4609, acc.: 78.91%] [G loss: 1.8987]
2/2 [==============================] - 0s 15ms/step
4809/5000 [D loss: 0.4550, acc.: 78.91%] [G loss: 1.7679]
2/2 [==============================] - 0s 9ms/step
4810/5000 [D loss: 0.4088, acc.: 78.12%] [G loss: 1.8858]
2/2 [==============================] - 0s 12ms/step
4811/5000 [D loss: 0.3952, acc.: 78.91%] [G loss: 1.8944]
2/2 [==============================] - 0s 9ms/step
4812/5000 [D loss: 0.3594, acc.: 82.03%] [G loss: 2.3521]
2/2 [=======

2/2 [==============================] - 0s 15ms/step
4879/5000 [D loss: 0.5807, acc.: 71.88%] [G loss: 1.7941]
2/2 [==============================] - 0s 14ms/step
4880/5000 [D loss: 0.4269, acc.: 79.69%] [G loss: 2.2057]
2/2 [==============================] - 0s 12ms/step
4881/5000 [D loss: 0.4030, acc.: 82.81%] [G loss: 2.4608]
2/2 [==============================] - 0s 21ms/step
4882/5000 [D loss: 0.3211, acc.: 84.38%] [G loss: 2.7260]
2/2 [==============================] - 0s 17ms/step
4883/5000 [D loss: 0.4101, acc.: 79.69%] [G loss: 2.2997]
2/2 [==============================] - 0s 17ms/step
4884/5000 [D loss: 0.4282, acc.: 80.47%] [G loss: 2.2455]
2/2 [==============================] - 0s 8ms/step
4885/5000 [D loss: 0.3918, acc.: 82.81%] [G loss: 2.4304]
2/2 [==============================] - 0s 7ms/step
4886/5000 [D loss: 0.3454, acc.: 86.72%] [G loss: 2.6107]
2/2 [==============================] - 0s 15ms/step
4887/5000 [D loss: 0.3219, acc.: 88.28%] [G loss: 2.5019]
2/2 [=======

2/2 [==============================] - 0s 26ms/step
4954/5000 [D loss: 0.4984, acc.: 77.34%] [G loss: 2.0376]
2/2 [==============================] - 0s 11ms/step
4955/5000 [D loss: 0.3523, acc.: 86.72%] [G loss: 2.9397]
2/2 [==============================] - 0s 20ms/step
4956/5000 [D loss: 0.5138, acc.: 80.47%] [G loss: 2.5571]
2/2 [==============================] - 0s 12ms/step
4957/5000 [D loss: 0.4336, acc.: 81.25%] [G loss: 2.7286]
2/2 [==============================] - 0s 11ms/step
4958/5000 [D loss: 0.3830, acc.: 82.03%] [G loss: 2.7883]
2/2 [==============================] - 0s 12ms/step
4959/5000 [D loss: 0.4505, acc.: 79.69%] [G loss: 2.5435]
2/2 [==============================] - 0s 10ms/step
4960/5000 [D loss: 0.5265, acc.: 77.34%] [G loss: 2.0755]
2/2 [==============================] - 0s 10ms/step
4961/5000 [D loss: 0.3467, acc.: 83.59%] [G loss: 2.3914]
2/2 [==============================] - 0s 9ms/step
4962/5000 [D loss: 0.4702, acc.: 77.34%] [G loss: 2.1383]
2/2 [======